# Power analysis attack on AES with custom SBox 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import serial.tools.list_ports as port_list
import sys
sys.path.append( '../AES_python' )
sys.path.append( '../sca_python' )

ports = list(port_list.comports())
for p in ports:
    print (p)

tested_sbox = "sbox_rijndael_lut"

#sbox_id = tested_sbox.replace("sbox_", "")
#project_file = "../../build/sca_test/sca_test_CW305_"+sbox_id+".cwp"
bitstream = r"../../hw/fpga/bitstream/polito_cw305_heep_cw305_heep_0.0.1.bit"
#bitstream = r"../../hw/fpga/bitstream/aes/aes_single_round/cw305_top_"+sbox_id+".bit"


/dev/ttyS4 - n/a
/dev/ttyS31 - n/a
/dev/ttyS30 - n/a
/dev/ttyS29 - n/a
/dev/ttyS28 - n/a
/dev/ttyS27 - n/a
/dev/ttyS26 - n/a
/dev/ttyS25 - n/a
/dev/ttyS24 - n/a
/dev/ttyS23 - n/a
/dev/ttyS22 - n/a
/dev/ttyS21 - n/a
/dev/ttyS20 - n/a
/dev/ttyS19 - n/a
/dev/ttyS18 - n/a
/dev/ttyS17 - n/a
/dev/ttyS16 - n/a
/dev/ttyS15 - n/a
/dev/ttyS14 - n/a
/dev/ttyS13 - n/a
/dev/ttyS12 - n/a
/dev/ttyS11 - n/a
/dev/ttyS10 - n/a
/dev/ttyS9 - n/a
/dev/ttyS8 - n/a
/dev/ttyS7 - n/a
/dev/ttyS6 - n/a
/dev/ttyS5 - n/a
/dev/ttyS3 - n/a
/dev/ttyS2 - n/a
/dev/ttyS1 - n/a
/dev/ttyS0 - n/a
/dev/ttyUSB5 - Quad RS232-HS - Quad RS232-HS
/dev/ttyUSB4 - Quad RS232-HS - Quad RS232-HS
/dev/ttyUSB3 - Quad RS232-HS - Quad RS232-HS
/dev/ttyUSB0 - FT232R USB UART - FT232R USB UART
/dev/ttyACM0 - Nano 33 BLE


In [2]:
print("bitstream: ", bitstream)

bitstream:  ../../hw/fpga/bitstream/polito_cw305_heep_cw305_heep_0.0.1.bit


In [ ]:
from picoscope import ps5000a
import chipwhisperer as cw
import time
from picosdk.discover import find_all_units


NameError: name 'cw305' is not defined

## Picoscope and CW305 initialization

In [4]:
from pico_api import PS5000aWrapper
from CW305_api import CW305Wrapper

try:
    # Initialize picoscope
    ps = PS5000aWrapper()
    ps.get_unitInfo()
    ps.scope_setup()
    # Initialize CW305
    cw305 = CW305Wrapper(ps, bitstream)
except ModuleNotFoundError as e:
    print(e)

DeviceNotFoundError: Could not find any devices on any drivers.

## Online phase : AES power traces capture

In [ ]:
from tqdm.notebook import tnrange
import chipwhisperer as cw
from Crypto.Cipher import AES
from AES_golden import AES_golden_model

project = cw.create_project(project_file, overwrite=True)

# Initialize key,text pair generator
ktp = cw.ktp.Basic()
key, text = ktp.next()

# Initialize the AES software emulated cipher
cipher = AES_golden_model()

# Each element of the key is converted to a 2-digit hex string 
formatted_key = ''.join(format(el, '02x') for el in key)
print("Key: ", [ hex(subkey) for subkey in key])

# Write the key to the CW305
cw305.set_key(key)
# Number of traces to capture
N = 2500      
# Dummy capture call due to bug of using AC coupling
cw305.capture_trace(text, project)

for i in tnrange(N, desc='Capturing traces'):
    response = cw305.capture_trace(text, project)
    # Sanity check with expected ciphertext
    formatted_pt = (format(el, '02x') for el in text)
    plaintext = [int(subbyte, 16) for subbyte in formatted_pt]
    assert (list(response) == list(cipher.encrypt(formatted_key, plaintext, tested_sbox))), "Incorrect encryption result!\nGot {}\nExp {}\n".format(list(response), list(text))
    key, text = ktp.next() 

project.save()
project.close()
# Disconnect CW305 and picoscope
cw305.dis()
ps.dis()


## Offline phase : attack on captured traces

In [ ]:
import os
import chipwhisperer.analyzer as cwa
import chipwhisperer as cw
import holoviews as hv
hv.extension('bokeh')
from analyzer.attack.aes.SBox_leakage_models import AES128SboxResistantLeakageModels
from analyzer.attack.aes.key_schedule import key_schedule_rounds
from analyzer.utils.sca_plots import sca_plot

sca_plt = sca_plot()
project = cw.open_project(project_file)

### Plotting power traces captured (40 samples overlapped)

In [ ]:

power_plt = sca_plt.power_traces_overlapped(ps.get_samplingInterval(), project.waves, finish=800)
power_plt.show()
# Ensure the Graphs directory exists
os.makedirs("Graphs", exist_ok=True)
power_plt.savefig("Graphs/power_traces_overlapped"+tested_sbox+".png")

### SNR evaluation : detect leakage points

In [ ]:
#leak_model = cwa.leakage_models.sbox_output
#project.traces.extend(project.traces)

# First round sbox output snr calculation
leak_model = AES128SboxResistantLeakageModels().FirstRound_ModifiedSbox_Output(tested_sbox)
snr_fr = cwa.calculate_snr(project.traces, leak_model=leak_model, db=False)
curve = hv.Curve(snr_fr).opts(height=600, width=600, color='orange', alpha=0.5)
# Last round state diff snr calculation
leak_model = AES128SboxResistantLeakageModels().LastroundStateDiff_ModifiedSbox(tested_sbox)
snr_lr = cwa.calculate_snr(project.traces, leak_model=leak_model, db=False)
curve *= hv.Curve(snr_lr).opts(height=600, width=600, color='blue', alpha=0.5)

curve.opts(width=900, height=600, show_legend=True, legend_position='right', xlabel='Sample', ylabel='SNR', title="SNR in time") 


### Hamming distance attack : state difference leakage model on last round  

In [ ]:
#Note : Chipwhisperer leakage model can be found in "cwa.leakage_models"

# Run CPA attack with the custom SBox leakage model
leak_class = AES128SboxResistantLeakageModels()
leak_model = leak_class.LastroundStateDiff_ModifiedSbox(tested_sbox)
attack = cwa.cpa(project, leak_model)
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cwa.get_jupyter_callback(attack, 10))
#print(results.best_guesses())

# Recover key
recv_lastroundkey = [kguess[0][0] for kguess in results.find_maximums()]
recv_key = key_schedule_rounds(recv_lastroundkey, 10, 0, tested_sbox)
print("Recovered key: ", [hex(subkey) for subkey in recv_key])
key=list(project.keys[0])
assert (key == recv_key), "Failed to recover encryption key!\nGot {}\nExp {}\n".format(recv_key, key)
print("Key recovery : Success!")

## Graphical Results : interpreting the attack

#### Plot SNR and captured power traces on the same insert

In [ ]:
leak_class = AES128SboxResistantLeakageModels()
leak_model = leak_class.LastroundStateDiff_ModifiedSbox(tested_sbox)
snr_vs_power_plt = sca_plt.snr_vs_traces_plot(ps.get_samplingInterval(), project, leak_model, finish=800)
snr_vs_power_plt.show()
# Ensure the Graphs directory exists
os.makedirs("Graphs", exist_ok=True)
snr_vs_power_plt.savefig("Graphs/PowerVsSNR_"+tested_sbox+".png")
snr_vs_power_plt.close(fig)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

def byte_to_color(idx):
  cmap = cm.get_cmap('cividis')  # Choose any colormap with 40 colors
  return cmap(idx / 40.0)

db_flag = False
# Create the figure and subplotscurve *= hv.Curve(snr_fr).opts(height=600, width=600, color='blue', alpha=0.5)
fig, (ax1, ax2) = plt.subplots(2, 1)  # 2 rows, 1 column
ax1.grid(axis='both', alpha=0.2)
ax2.grid(axis='both', alpha=0.2)

finish_interval = 800
xticks_samples = range(len(traces[0]))
xfocus_window = xticks_samples[:finish_interval]
xtick_us = [x*actualSamplingInterval*1E6 for x in xfocus_window]
plt.title("Power traces and SNR")

# Plot the power traces
ax1.set_xlabel("Time (us)")
ax1.set_ylabel("Voltage (mV)")
for i in range(40):
    ax1.plot(xtick_us, (traces[i][:finish_interval]*1000), color=byte_to_color(i), alpha=0.5)
ax1.legend()

# Plot the SNR
ax2.set_xlabel("Time (us)")
ax2.set_ylabel("SNR")
# snr on the first round sbox output leakage model (between plaintext and sbxo output)
leak_model = cwa.leakage_models.sbox_output
snr_so = cwa.calculate_snr(project.traces, leak_model=leak_model, db=db_flag)
ax2.plot(xtick_us, snr_so[:finish_interval], color='blue', label='First round - Sbox out', alpha=0.5)
# snr on the last round state diff leakage model (between the last round state and the ciphertext)
leak_model = cwa.leakage_models.last_round_state_diff
snr_lrsd = cwa.calculate_snr(project.traces, leak_model=leak_model, db=db_flag)
ax2.plot(xtick_us, snr_lrsd[:finish_interval], color='orange', label='Last round - State diff')
ax2.legend(title=f"leakage model", fontsize=6, loc="upper right")

# Adjust layout (optional)
plt.tight_layout()
plt.show()
plt.savefig("Graphs/PowerVsSNR.png")
plt.close(fig)

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.operation import decimate
import pandas as pd, numpy as np
plot_data = cwa.analyzer_plots(results)

### - Calculated correlation at each point in time for the correct key

In [ ]:
def byte_to_color(idx):
    return hv.Palette.colormaps['Category20'](idx/16.0)

a = []
b = []
hv.extension('bokeh')

# display the calculated correlation at each point in time for the coreect key
for key_byte in range(0, 16):
    data = plot_data.output_vs_time(key_byte)
    a.append(np.array(data[1]))
    b.append(np.array(data[2]))
    b.append(np.array(data[3]))

pda = pd.DataFrame(a).transpose().rename(str, axis='columns')
pdb = pd.DataFrame(b).transpose().rename(str, axis='columns')   
curve = hv.Curve(pdb['0'], "Sample").options(color='black')

for key_byte in range(1, 16):
    curve *= hv.Curve(pdb[str(key_byte)]).options(color=byte_to_color(key_byte))

for key_byte in range(0, 16):
    curve *= hv.Curve(pda[str(key_byte)]).options(color=byte_to_color(key_byte))
curve.opts(width=900, height=600)
#decimate(curve.opts(width=900, height=600)) 

## PGE vs Traces

###### with matplot lib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
"""
Plots the PGE trends for all the 16 correct key guesses with matplotlib
"""

pges = [plot_data.pge_vs_trace(i) for i,_ in enumerate(key)]
fig, ax1 = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=[14,10])
plt.grid(which='major', axis='both', alpha=0.2)

x_axis = plot_data.pge_vs_trace(0)[0]
step = 100
for i, bnum in enumerate(key):
    plt.plot(pges[i][0], pges[i][1], linewidth=1, label=f"Byte #{i}")
plt.plot(pges[i][0], [10 for _ in range(len(pges[i][0]))], linewidth=1, linestyle="dashed",  color="black", label=f"max(PGE) < {10}")
    
plt.legend(title=f"Known Key", fontsize=12, loc="upper right")
plt.title("AES SBOX Freyre 2 - PGE", fontsize=18)
ax1.set_xticks(range(0, x_axis[-1], step))
# clip_min_y = 0
# clip_max_y = 300
# ax1.set_yticks(list(range(clip_min_y, clip_max_y+1, 10)))
ax1.set_ylabel('Partial Guessing Entropy (PGE)', fontsize=16)
ax1.set_xlabel('Traces', fontsize=16)
# plt.show()
plt.savefig("Graphs/PGE_hussain_6.png")
plt.close(fig)

In [ ]:
ret = plot_data.pge_vs_trace(0)
curve = hv.Curve((ret[0], ret[1], "Traces Used in Calculation", "Partial Guessing Entropy of Byte"))

for bnum in range(1,16):
    ret = plot_data.pge_vs_trace(bnum)
    curve *= hv.Curve((ret[0], ret[1])).opts(color=byte_to_color(bnum))
curve.opts(width=900, height=600)

####  Correlation traces

In [ ]:
plot_data.plot_corr_vs_trace()

In [ ]:
""" corr_vs_trace(self, bnum)
Gets plot data of correlation for each possible value of subkey
        vs. trace number for a given subkey"""
bnum
data = results.maxes_list
# take per byte 0 the the correlation values of each of the 256 subkey guesses, ranked by correlation. Returned this list per each trace (traces intervals of 25)
maxdata_byte0=data[0]
maxdata_byte0_after25traces = maxdata_byte0[100]
#print("Subkey guesses for byte 0 after 25 traces ranked in order of correlation:\n", maxdata_byte0_after25traces)

tlist = []
for m in maxdata_byte0:
    # append the trace value (number) to the list. Maxdata_byte0 is a list of dictionaries, each dictionary contains the trace number and the maxes for each subkey guess 
    # each guess is formatted as (#subkey_value, ?, correlation_value)
    tlist.append(m['trace'])
# maxlist is a 2d array, #rows = possible subkey values (=256, since byte in aes), #columns = #traces/25
maxlist = np.zeros((len(results.diffs[bnum]), len(tlist)))
for i, m in enumerate(maxdata_byte0):
    # where j = len(results.diffs[bnum]) = 256, since subkey is byte in aes
    for j in range(0, len(results.diffs[bnum])):
        # Took each subkey guess(=256), at trace interval m, and stored the correlation value in the maxlist 
        maxlist[m['maxes'][j][0], i] = m['maxes'][j][2]

# tlist : trace number (0-3000, step =25)
# maxlist : sukey guesses correlation value ranked,for byte 0, at each trace interval
corrVsTrace_byte0 = [tlist, maxlist]

In [ ]:
# diffs, one per each subkey (=16)
# each diff is a list of 256 elements, one per each possible byte value
# each element is a list of 1260 elements, one per each trace --> correlation value each trace interval 
print(len(results.diffs[0]))

##### Correlation of best key guess and next best one with holoview


In [ ]:
import holoviews as hv
from holoviews.operation import decimate
import pandas as pd, numpy as np
hv.extension('bokeh')
def byte_to_color(idx):
    return hv.Palette.colormaps['Category20'](idx/16.0)

# 16 subkey
bnum_it = range(16)

curve = None
for bnum in bnum_it: # per each subkey byte 
    # data store the correlation values (ranked) for subkey guesses of byte 'bnum', at each trace intervalax1.set_xticks(range(0, x_axis[-1], step))
    data = np.array(plot_data.corr_vs_trace(bnum)[1])
    # trace intervals (first  of corr_vs_trace return value)
    xrangelist = plot_data.corr_vs_trace(bnum)[0]
    key = results.known_key[bnum]
    #print("#trace intervals: ", len(xrangelist), ",len correlation array: ",len(data[key]), " @byte bnum: ",bnum)
    #print("correlation trend:\n", data[key])
    wrong_results = data[[i != key for i in range(256)]]
    plt = hv.Curve((xrangelist, data[key])).opts(color=byte_to_color(bnum))
   
    if curve is None:
        curve = plt
    else:
        curve *= plt
    #print("next wrong subkey correlation trend guess for byte", bnum, ":\n", np.amax(wrong_results, 0))
    # took correlation value of first wrong subkey guess for byte 'bnum', at each trace interval
    curve *= hv.Curve((xrangelist, np.amax(wrong_results, 0))).opts(color=byte_to_color(bnum), alpha=0.5, line_dash='dotted')


curve.opts(width=900, height=600, show_legend=True, legend_position='right', xlabel='Traces', ylabel='Correlation', title="Correlation Vs. Traces")   

#### Correlation vs Traces with matplot lib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def byte_to_color(idx):
    return hv.Palette.colormaps['Category20'](idx/16.0)
"""
Plots the various correlations of both the correct key guesses and the wrong ones
"""
# 16 subkeyCW305_leakage_model
bnum_it = range(16)

fig, ax1 = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=[18,12])
plt.grid(which='major', axis='both', alpha=0.2)

# Plot correlation trends of wrong key guesses (
for bnum in bnum_it:
    data = np.array(plot_data.corr_vs_trace(bnum)[1])
    xrangelist = plot_data.corr_vs_trace(bnum)[0]
    wrong_results = data[[i != key for i in range(256)]]
    ax1.plot(xrangelist, np.amax(wrong_results, 0), linewidth=2, linestyle ="dotted", color=byte_to_color(bnum))

# Plot correct key_guesses on top of wrong ones
for bnum in bnum_it:
    data = np.array(plot_data.corr_vs_trace(bnum)[1])
    xrangelist = plot_data.corr_vs_trace(bnum)[0]
    key = results.known_key[bnum]
    wrong_results = data[[i != key for i in range(256)]]
    ax1.plot(xrangelist, data[key], linewidth=1, label=f"Byte #{bnum}", color=byte_to_color(bnum))


ax1.legend(title=f"Known Key", fontsize=12, loc="upper right")
plt.title(" AES SBOX Freyre 2 - Correlations", fontsize=18)

step = 200
ax1.set_xticks(range(0, xrangelist[-1], step))
ax1.set_ylabel('Correlation', fontsize=16)
ax1.set_xlabel('Traces', fontsize=16)

#plt.show()
plt.savefig("Graphs/correlations_Freyre_2.png")
plt.close(fig)

In [ ]:
plot = plot_data.corr_vs_trace(0)

### Performance comparison of modified SBox vs standard SBox (Rijndael)

##### Run attack on standard S-Box (Rijndael) and tested one

In [ ]:
import chipwhisperer.analyzer as cwa
import chipwhisperer as cw

# Run attack on Rijndael Sbox
project_file_Rijndael = "../../build/sca_test/sca_test_CW305_Rijndael.cwp"
project = cw.open_project(project_file_Rijndael)
key_Rijndael=list(project.keys[0])
attack = cwa.cpa(project, cwa.leakage_models.last_round_state_diff)
cb = cwa.get_jupyter_callback(attack)
results_rijandel = attack.run(cwa.get_jupyter_callback(attack, 10))
project.close()

# Run the attack on tested sbox
project = cw.open_project(project_file)
key_Freyre_2=list(project.keys[0])
attack = cwa.cpa(project, cwa.leakage_models.last_round_state_diff_freyre_2)
cb = cwa.get_jupyter_callback(attack)
results_Freyre_2 = attack.run(cwa.get_jupyter_callback(attack, 10))
project.close()

##### Plot PGEs trend and correlation trend for both S-Boxes (side by side)

In [ ]:
"""
Comparion of PGE trend and correlation trend (of next best guess) between the two sboxes 
"""
import holoviews as hv
hv.extension('bokeh')

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D  # For custom legends
import numpy as np
import chipwhisperer.analyzer as cwa

# Create the figure and subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)  # 2 rows, 2 columns
ax1.grid(axis='both', alpha=0.2)
ax2.grid(axis='both', alpha=0.2)
ax3.grid(axis='both', alpha=0.2)
ax4.grid(axis='both', alpha=0.2)

ax1.set_title('Rijndael S-box',fontweight='bold')
ax2.set_title('Freyre 2 S-box',fontweight='bold')

def byte_to_color(idx):
    return hv.Palette.colormaps['Category20'](idx/16.0)

bnum_it = range(16)

# Plot object for both Sboxes
plot_data_Rijndael = cwa.analyzer_plots(results_rijandel)
plot_data_Freyre_2 = cwa.analyzer_plots(results_Freyre_2)

"""
Plot the PGE trend for both the sboxes
"""
pges_Rijndael = [plot_data_Rijndael.pge_vs_trace(i) for i,_ in enumerate(key_Rijndael)]
pges_Freyre_2 = [plot_data_Freyre_2.pge_vs_trace(i) for i,_ in enumerate(key_Freyre_2)]

pge_xrangelist = plot_data_Rijndael.pge_vs_trace(0)[0]

# Plot PGE trends for Rijndael Sbox
for i, bnum in enumerate(key_Rijndael):
    ax1.plot(pges_Rijndael[i][0], pges_Rijndael[i][1], linewidth=0.5, label=f"Byte #{i}")
ax1.plot(pges_Rijndael[i][0], [10 for _ in range(len(pges_Rijndael[i][0]))], linewidth=1, linestyle="dashed",  color="black", label=f"max(PGE) < {10}")
x_lim_right = ax1.get_xlim()[1]-100  # Get the right limit of the x-axis and leave 100 as offset
y_position_above_line = 10 + 0.5  # Slightly above the line, adjust 0.5 or as needed
ax1.text(x_lim_right, y_position_above_line, 'max(PGE) < 10', fontsize=5, color='black', ha='right', va='bottom')
ax1.set_ylabel('Partial Guessing Entropy (PGE)', fontsize=16)
ax1.set_xlabel('Traces', fontsize=16)
step = 500
ax1.set_xticks(range(0, pge_xrangelist[-1], step))
ax1.set_ylabel('PGE', fontsize=16)
ax1.set_xlabel('Traces', fontsize=16)



# Plot PGE trends for Freyre 2 Sbox
for i, bnum in enumerate(key_Freyre_2):
    ax2.plot(pges_Freyre_2[i][0], pges_Freyre_2[i][1], linewidth=0.5, label=f"Byte #{i}")
ax2.plot(pges_Rijndael[i][0], [10 for _ in range(len(pges_Freyre_2[i][0]))], linewidth=1, linestyle="dashed",  color="black", label=f"max(PGE) < {10}")
x_lim_right = ax2.get_xlim()[1]-100  # Get the right limit of the x-axis and leave 100 as offset
y_position_above_line = 10 + 0.5  # Slightly above the line, adjust 0.5 or as needed
ax2.text(x_lim_right, y_position_above_line, 'max(PGE) < 10', fontsize=5, color='black', ha='right', va='bottom')
ax2.set_ylabel('PGE', fontsize=16)
ax2.set_xlabel('Traces', fontsize=16)
step = 500
pge_xrangelist = plot_data_Freyre_2.pge_vs_trace(0)[0]
ax2.set_xticks(range(0, pge_xrangelist[-1], step))
ax2.set_ylabel('PGE', fontsize=16)
ax2.set_xlabel('Traces', fontsize=16)

"""
Plot the correlation trend for both the sboxes. 
In the correlation plot is reported the trend of the next best guess for each byte
"""
# Create custom legend items
legend_items = [Line2D([0], [0], color='black', linewidth=1, linestyle='--', label='Wrong subkeys'),
                Line2D([0], [0], color='black', linewidth=1, linestyle='-', label='Correct subkeys')]

# Plot correlation trends for Rijndael Sbox
corr_xrange_list = plot_data_Rijndael.corr_vs_trace(0)[0]
# -- Next best wrong key guess for each byte
for bnum in bnum_it:
    data = np.array(plot_data_Rijndael.corr_vs_trace(bnum)[1])
    key = results_rijandel.known_key[bnum]
    wrong_results = data[[i != key for i in range(256)]]
    ax3.plot(corr_xrange_list , np.amax(wrong_results, 0), linewidth=1, linestyle ="dotted", color=byte_to_color(bnum))
 # -- Correct key guess for each byte
for bnum in bnum_it:
    data = np.array(plot_data_Rijndael.corr_vs_trace(bnum)[1])
    key = results_rijandel.known_key[bnum]
    wrong_results = data[[i != key for i in range(256)]]
    ax3.plot(corr_xrange_list , data[key], linewidth=1, label=f"Byte #{bnum}", color=byte_to_color(bnum))


# Add the custom legend to the plot
# Assuming ax1 is the axis you want to add the legend to. Replace ax1 with the appropriate axis variable name if different.
ax3.legend(handles=legend_items, title='Subkey Guesses', loc='upper right', fontsize='small')
ax3.set_xticks(range(0, corr_xrange_list[-1], step))
ax3.set_ylabel('Correlation', fontsize=13)
ax3.set_xlabel('Traces', fontsize=12)

# Plot correlation trends for Freyre 2 Sbox
# -- Next best wrong key guess for each byte
for bnum in bnum_it:
    data = np.array(plot_data_Freyre_2.corr_vs_trace(bnum)[1])
    key = results_Freyre_2.known_key[bnum]
    wrong_results = data[[i != key for i in range(256)]]
    ax4.plot(corr_xrange_list , np.amax(wrong_results, 0), linewidth=1, linestyle ="dotted", color=byte_to_color(bnum))
 # -- Correct key guess for each byte
for bnum in bnum_it:
    data = np.array(plot_data_Freyre_2.corr_vs_trace(bnum)[1])
    key = results_Freyre_2.known_key[bnum]
    wrong_results = data[[i != key for i in range(256)]]
    ax4.plot(corr_xrange_list , data[key], linewidth=1, label=f"Byte #{bnum}", color=byte_to_color(bnum))

ax4.legend(handles=legend_items, title='Subkey Guesses', loc='upper right', fontsize='small')
ax4.set_xticks(range(0, corr_xrange_list[-1], step))
ax4.set_ylabel('Correlation', fontsize=13)
ax4.set_xlabel('Traces', fontsize=12)

for ax in fig.get_axes():
    ax.label_outer()

# Adjust layout 
#plt.tight_layout()
#plt.show()
#plt.savefig("Graphs/sbox_comparison.pdf", dpi=1200)
plt.savefig("Graphs/sbox_comparison.pdf")
plt.close(fig)